# RNN

# 1.词嵌入

In [3]:
import torch
from torch import nn,optim
from torch.nn import functional as F

* nn.Embedding(num_embeddings,embedding_dim,padding_idx = None,max_norm = None,norm_type = 2.0,scale_grad_by_freq = False,sparse = False,_weight = None)

# 参数：
* num_embeddings(int):词典的尺寸大小
* embedding_dim:嵌入向量的维度
* padding_idx(int，可选):比如，输入长度为100，但是每次的句子长度不一样，后面就需要统一的数字填充，这里就是指定这个数字
* max_norm(float,可选)：最大范数，如果嵌入向量的范数超过了这个界限，就要进行再归一化
* norm_type(float,可选)：指定利用什么范数计算，默认为2范数

In [4]:
word_to_idx = {'hello':0,'world':1}
# 得到目标单词索引
lookup_tensor = torch.tensor([word_to_idx['hello']],dtype = torch.long)
embeds = nn.Embedding(len(word_to_idx),5)

In [5]:
hello_embed = embeds(lookup_tensor)

In [6]:
hello_embed

tensor([[ 0.5902, -0.6656, -0.4503, -0.7510, -1.7887]],
       grad_fn=<EmbeddingBackward>)

# 2.nn.RNN

* nn.RNN(input_size,hidden_size,num_layers = 1,nonliearity = tanh,bias = True,batch_first = False,dropout = 0,bidirectional = False)

# 参数
* input_size:embedding_dim
* hidden_size：隐藏层神经元个数
* num_layers :几层
* nonlinearity:激活函数
* bias:是否使用偏置
* batch_first:默认为True,即（seq_len,batch,embedding_dim）
* dropout:默认为0
* bidirectional:默认为False

In [7]:
rnn = nn.RNN(100,10)


In [9]:
rnn._parameters.keys()

odict_keys(['weight_ih_l0', 'weight_hh_l0', 'bias_ih_l0', 'bias_hh_l0'])

In [12]:
print(rnn.weight_hh_l0.shape,rnn.weight_ih_l0.shape)

torch.Size([10, 10]) torch.Size([10, 100])


In [14]:
print(rnn.bias_hh_l0.shape,rnn.bias_ih_l0.shape)

torch.Size([10]) torch.Size([10])


# 2.2 forward前向传播
* x:[seq_len,batch,embedding_dim]一次性将所有时刻特征喂入
* h0/ht:[num_layers,batch,hidden_len]
* out:[seq_len,batch,hidden_size]:每一时刻空间上最后一层的输出

In [15]:
rnn = nn.RNN(100,10,num_layers = 5)

In [16]:
x = torch.randn(10,3,100)#[seq_len,batch,embedding_dim]

In [17]:
# 传入x以及h_0:[num_layers,batch_size,hidden_size]
out,h = rnn(x,torch.zeros(5,3,10))

In [18]:
out.shape

torch.Size([10, 3, 10])

In [19]:
h.shape

torch.Size([5, 3, 10])

# 3.nn.RNNCell()
* xt:[batch,embeding_dim]表示当前时刻的输入
* ht-1:[num_layers,batch,hidden]:前一时刻的单元输出，ht是下一时刻的单元输入

In [22]:
# embedding_dim=100,hidden_size = 10,batch = 3,seq_ len = 20
cell = nn.RNNCell(100,10)
h1 = torch.zeros(3,10)
x = torch.randn(20,3,100)
for xt in x:
    h1 = cell(xt,h1)
print(h1.shape)

torch.Size([3, 10])


In [23]:
h1

tensor([[-0.9363, -0.5713,  0.6803, -0.8702,  0.0882, -0.1025,  0.7673,  0.5834,
         -0.7532, -0.9827],
        [-0.9115, -0.9989,  0.1442, -0.0086,  0.7014, -0.9945, -0.9983, -0.9958,
         -0.8351, -0.3380],
        [ 0.9958,  0.9598, -0.0167,  0.9660, -0.6955, -0.9970, -0.1947,  0.9654,
          0.1267, -0.5725]], grad_fn=<TanhBackward>)

In [25]:
# 多层RNN
h1  = torch.zeros(3,30)
h2 = torch.zeros(3,10)
cell1 = nn.RNNCell(100,30)
cell2 = nn.RNNCell(30,10)
x = torch.randn(10,3,100)
for xt in x:
    h1 = cell1(xt,h1)
    h2 = cell2(h1,h2)

In [ ]:
print(h1.shape)